In [1]:
import geopandas
import folium

/tmp/ipykernel_162/2797155997.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas


In [2]:
df = geopandas.read_file("EWR Alignment Ox2CamSouth ln.shp")
print(df)

   objectid name    st_length_  \
0         1  CS3  65926.744941   
1         2  CS2  27395.409475   
2         3  CS1  50417.735543   

                                            geometry  
0  MULTILINESTRING Z ((544377.408 251506.330 0.00...  
1  LINESTRING Z (486936.480 233752.200 0.000, 486...  
2  MULTILINESTRING Z ((468677.990 225730.870 0.00...  


In [3]:
df.crs

In [4]:
from pyproj import CRS

In [5]:
crs = CRS('EPSG:27700')

In [6]:
df.crs = crs

In [14]:
df.to_crs("EPSG:4326",inplace=True)

In [15]:
df = df.explode(index_parts=True)
print(df)

         objectid name    st_length_  \
0 0 0 0         1  CS3  65926.744941   
  1 0 0         1  CS3  65926.744941   
  2 0 0         1  CS3  65926.744941   
  3 0 0         1  CS3  65926.744941   
1 0 0 0         2  CS2  27395.409475   
2 0 0 0         3  CS1  50417.735543   
  1 0 0         3  CS1  50417.735543   

                                                  geometry  
0 0 0 0  LINESTRING Z (0.10847 52.14300 0.00000, 0.1067...  
  1 0 0  LINESTRING Z (0.10847 52.14300 0.00000, 0.1111...  
  2 0 0  LINESTRING Z (-0.48467 52.14112 0.00000, -0.48...  
  3 0 0  LINESTRING Z (-0.48422 52.15685 47.99900, -0.4...  
1 0 0 0  LINESTRING Z (-0.73517 51.99543 0.00000, -0.73...  
2 0 0 0  LINESTRING Z (-1.00265 51.92588 0.00000, -0.99...  
  1 0 0  LINESTRING Z (-1.00266 51.92588 0.00000, -1.01...  


In [16]:
m = folium.Map(control_scale=True)
feature_dict={a:folium.FeatureGroup(name=a) for a in list(df['name'].unique())}
for r in df.iterrows():
	location_tmp = list([(lat, lon) for lon, lat, alt in r[1]['geometry'].coords])
	folium.features.PolyLine(locations=location_tmp).add_to(feature_dict[r[1]['name']])
for fgs in feature_dict.values():
         fgs.add_to(m)
#folium.GeoJson(df.to_json(to_wgs84=True)).add_to(m)
folium.LayerControl().add_to(m)
m.fit_bounds(m.get_bounds())
m.save('index.html')

In [17]:
m